In [1]:
#initialize parametrized variables

experimentName = ''
runID = ''


In [2]:
# Parameters
runID = "744963e3a6754ac482e5dd8bc4b74111"
experimentName = "dummyabc"


In [3]:
import jupyter_client
jupyter_client.kernelspec.find_kernel_specs()

{'python3': '/home/pedro/anaconda3/envs/mlflow-7ce0d1002c6479899f7ec502c9b84f5ce61312e3/share/jupyter/kernels/python3'}

In [4]:
import pandas as pd
import numpy as np

import mlflow, mlflow.sklearn
from mlflow.pyfunc import PythonModel

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [5]:
#sets the MLflow experiment and active run
mlflow.set_experiment(experiment_name=experimentName)
mlflow.start_run(runID)

<ActiveRun: >

In [6]:
#data loading from storage to be handled by project_name
data = pd.read_csv('/home/pedro/Documentos/work/Jupyter_Python/SuperTopicModeling/BERT-service/dataset/smart-intents.csv', header=None, names=['Input','Intent'])
data = data.sample(1500)
data

,Input,Intent
586,Ich suche 3D-Modelle von smart.,DE_apps_experience
1606,Sag mir etwas über den Chef von der Abteilung ...,DE_brand_daimlerboard_wilfried_porth
3577,Wer ist deine Freundin?,DE_chitchat_bot_relationship
2965,Ist es möglich an einer Wallbox zu laden?,DE_charging_possibilities_Wallbox
2705,Stehen mehrere Auswahlmöglichkeiten zur Auswah...,DE_charging_possibilities
...,...,...
1740,Wie lang geht die Garantie?,DE_brand_guarantee
3343,Liest du gerne Zeitschriften?,DE_chitchat_bot_book
2237,Was ist der Radstand vom smart?,DE_car_characteristics_wheelbase
1991,Was steckt im Begriff smart?,DE_brand_smart_naming


In [7]:
class STMPredProba(PythonModel):
    
    def __init__(self, n=0):
        self._n = n
        self._model = None
        
        
    def fit(self, df):
        from vectorizer import vectorize
        
        df['sentence_vector'] = vectorize(df['Input'])
        X = pd.DataFrame(list(df['sentence_vector']), index=df.index)
        y= df['Intent']
        
        #defining pipe model
        pipeline = Pipeline([('scaler',MinMaxScaler()),
                             ('model', LogisticRegression(C=1,
                                                          solver='saga',
                                                          multi_class='multinomial',
                                                          max_iter=10000))]) 
        #constructing accuracy
        try: #initially can not be done when classes are unitary..
            X_train, X_test, y_train, y_test = train_test_split(X,
                                                                y,
                                                                test_size=0.2,
                                                                stratify = label_series) # to be contructed from previous predictions and respective new labels to avoid retraining
            pipeline.fit(X = X_train, y = y_train) #constructing the model to accuracy -> to be cut out: see above
            acc = pipeline.score(X = X_test,y = y_test)
            print(acc)
            mlflow.log_metric('accuracy',acc)
        except:
            pass
        # training the model to be served
        pipeline.fit(X, y)
        #store the MLflow model 
        mlflow.sklearn.log_model(pipeline,'SupTopModel')
        self._model = pipeline
        return None

    def predict(self, df, n=0):#, pipe_model, n=0):
       
        df['sentence_vector'] = self.vectorize(df['Input'])
        vector_df = pd.DataFrame(list(df['sentence_vector']), index=df.index)        
        prob_df = pd.DataFrame(self._model.predict_proba(vector_df), index=vector_df.index, columns=self._model.classes_)
        
        if n ==0:
            return prob_df
        else:
            top_n = np.argpartition(prob_df.transpose(), -self._n, axis=0)[-self._n:].apply(lambda x:
                                                                                  prob_df.transpose().index.values[
                                                                                      x]).transpose()
            top_n.columns = ["{}".format(i+1) for i in range(self._n)]
            prob_matrix = pd.DataFrame()
            for col in top_n.columns:
                prob_matrix[col] = prob_df.apply(lambda row: row[top_n[col][row.name]], axis=1)
            top_prob = pd.merge(top_n, prob_matrix, right_index=True, left_index=True)
            tuple_class_prob = pd.DataFrame()
            for i in range(1, self._n+1): #constructs the tuple table
                runner_x = '_'.join([str(i), 'x'])
                runner_y = '_'.join([str(i), 'y'])
                tuple_class_prob[i] = pd.DataFrame([top_prob[runner_x], top_prob[runner_y]]).transpose().apply(tuple,
                                                                                                           axis=1)
            tuple_class_prob.columns = [f"top{i+1}"+str(i) for i in range(1, self._n+1)]
            for i in range(len(tuple_class_prob)): #sorts the tuple table
                list_to_sort=tuple_class_prob.iloc[i].tolist()
                list_to_sort.sort(key=lambda x: -x[1])
                tuple_class_prob.iloc[i]=list_to_sort
            return tuple_class_prob

In [8]:
MLflow_model = STMPredProba()

MLflow_model.fit(data)

In [9]:
from bert_serving.client import BertClient


bc = BertClient(show_server_config=True , ignore_all_checks=True, port=5555, ip='localhost')


bc.encode(['hello my name is pedro'])

array([[ 6.79337680e-01,  6.84069097e-01,  5.73804617e-01,
         2.94803202e-01, -2.60168403e-01,  3.30935568e-01,
        -6.01355910e-01,  4.37773526e-01,  1.17255598e-01,
        -1.66149139e-01,  5.11813581e-01, -5.04456520e-01,
        -3.59423429e-01,  2.65473604e-01,  7.56017327e-01,
        -4.49600816e-01, -4.09083635e-01,  7.01568186e-01,
        -3.74411225e-01, -6.95729077e-01,  7.93639243e-01,
         3.21136951e-01, -1.23016620e+00, -4.00343657e-01,
         1.71914518e-01,  3.70896488e-01, -1.71218425e-01,
        -5.43357134e-01, -6.76492214e-01,  1.64480388e-01,
        -2.23036662e-01,  6.90588593e-01,  3.05051923e-01,
         4.95595574e-01,  1.70528382e-01,  1.01708949e+00,
        -5.14387846e-01, -4.59431022e-01, -6.02292717e-01,
        -7.11721122e-01, -3.15179467e-01,  5.10206997e-01,
        -6.66227341e-01,  6.95019245e-01,  1.33225167e+00,
         1.97900146e-01,  2.35533819e-01, -3.21643472e-01,
        -5.00770211e-01, -1.53736889e+00,  2.03108922e-0